<a href="https://colab.research.google.com/github/chang2694/Intrusion_detection_system/blob/main/IDS_using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [321]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/IDS NSL-KDD

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/IDS NSL-KDD


In [322]:
import numpy as np
import pandas as pd
from sklearn import preprocessing,model_selection
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models


In [323]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
train =  pd.read_csv('NSL_KDD_Train.csv', names = col_names)
test =  pd.read_csv('NSL_KDD_Test.csv', names = col_names)

In [324]:
df = pd.concat([train, test], ignore_index = True)
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148512,0,tcp,smtp,SF,794,333,0,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,normal
148513,0,tcp,http,SF,317,938,0,0,0,0,...,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,normal
148514,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,back
148515,0,udp,domain_u,SF,42,42,0,0,0,0,...,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [325]:
df.loc[df['label'] != 'normal', 'label'] = 'attack'
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,attack
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148512,0,tcp,smtp,SF,794,333,0,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,normal
148513,0,tcp,http,SF,317,938,0,0,0,0,...,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,normal
148514,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,attack
148515,0,udp,domain_u,SF,42,42,0,0,0,0,...,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [326]:
df.nunique(axis = 0).sort_values(ascending= False)

dst_bytes                      10401
src_bytes                       3601
duration                        3424
srv_count                        512
count                            512
dst_host_srv_count               256
dst_host_count                   256
dst_host_srv_serror_rate         101
dst_host_same_srv_rate           101
diff_srv_rate                    101
same_srv_rate                    101
dst_host_serror_rate             101
dst_host_same_src_port_rate      101
dst_host_diff_srv_rate           101
dst_host_rerror_rate             101
dst_host_srv_rerror_rate         101
serror_rate                       99
rerror_rate                       98
num_compromised                   96
srv_rerror_rate                   95
srv_serror_rate                   94
num_root                          91
srv_diff_host_rate                87
dst_host_srv_diff_host_rate       75
service                           70
num_file_creations                36
hot                               29
f

In [327]:
df = df.drop(['num_outbound_cmds', 'dst_bytes', 'src_bytes', 'duration'], axis = 1)
df

,protocol_type,service,flag,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,tcp,ftp_data,SF,0,0,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,udp,other,SF,0,0,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,tcp,private,S0,0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,attack
3,tcp,http,SF,0,0,0,0,0,1,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,tcp,http,SF,0,0,0,0,0,1,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148512,tcp,smtp,SF,0,0,0,0,0,1,0,...,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,normal
148513,tcp,http,SF,0,0,0,0,0,1,0,...,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,normal
148514,tcp,http,SF,0,0,0,2,0,1,1,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,attack
148515,udp,domain_u,SF,0,0,0,0,0,0,0,...,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [328]:
obj = []
for col_names in df.columns:
  if df[col_names].dtypes == 'object':
    obj.append(col_names)
obj.pop()
obj

['protocol_type', 'service', 'flag']

In [329]:
df_enc = df[obj]

In [330]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148517 entries, 0 to 148516
Data columns (total 38 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   protocol_type                148517 non-null  object 
 1   service                      148517 non-null  object 
 2   flag                         148517 non-null  object 
 3   land                         148517 non-null  int64  
 4   wrong_fragment               148517 non-null  int64  
 5   urgent                       148517 non-null  int64  
 6   hot                          148517 non-null  int64  
 7   num_failed_logins            148517 non-null  int64  
 8   logged_in                    148517 non-null  int64  
 9   num_compromised              148517 non-null  int64  
 10  root_shell                   148517 non-null  int64  
 11  su_attempted                 148517 non-null  int64  
 12  num_root                     148517 non-null  int64  
 13 

In [331]:
oneHot_enc = preprocessing.OneHotEncoder()
df_enc = oneHot_enc.fit_transform(df_enc)
df_enc.toarray()

array([[0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [332]:
df.index.is_unique

True

In [333]:
encoded = pd.DataFrame(df_enc.toarray(), columns=oneHot_enc.get_feature_names_out(obj))
df = df.drop(columns = obj)
df = pd.concat([df, encoded], axis = 1)

In [334]:
label_enc = preprocessing.LabelEncoder()
label_df = label_enc.fit_transform(df['label'])
label_df

array([1, 1, 0, ..., 0, 1, 0])

In [335]:
df = df.drop(columns='label')
scaler = preprocessing.Normalizer()
df = scaler.fit_transform(df)

In [336]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df, label_df, test_size=0.2, random_state=42)

In [337]:
model = models.Sequential()
model.add(layers.Dense(100, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(2, activation='softmax'))

In [338]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [339]:
y_train = pd.get_dummies(y_train)*1
y_train = y_train.values

In [340]:
y_test = pd.get_dummies(y_test)*1
y_test = y_test.values

In [341]:
y_train

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]], dtype=uint8)

In [342]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
3713/3713 [==============================] - 23s 4ms/step - loss: 0.1594 - accuracy: 0.9373
Epoch 2/100
3713/3713 [==============================] - 15s 4ms/step - loss: 0.0818 - accuracy: 0.9695
Epoch 3/100
3713/3713 [==============================] - 16s 4ms/step - loss: 0.0654 - accuracy: 0.9761
Epoch 4/100
3713/3713 [==============================] - 16s 4ms/step - loss: 0.0583 - accuracy: 0.9789
Epoch 5/100
3713/3713 [==============================] - 15s 4ms/step - loss: 0.0574 - accuracy: 0.9795
Epoch 6/100
3713/3713 [==============================] - 15s 4ms/step - loss: 0.0518 - accuracy: 0.9812
Epoch 7/100
3713/3713 [==============================] - 15s 4ms/step - loss: 0.0497 - accuracy: 0.9819
Epoch 8/100
3713/3713 [==============================] - 16s 4ms/step - loss: 0.0478 - accuracy: 0.9829
Epoch 9/100
3713/3713 [==============================] - 15s 4ms/step - loss: 0.0456 - accuracy: 0.9831
Epoch 10/100
3713/3713 [==============================] - 15s 4m

In [343]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

929/929 [==============================] - 3s 3ms/step - loss: 0.0846 - accuracy: 0.9911
Test accuracy: 0.9910786151885986
